In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [3]:
test.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [8]:
df = pd.DataFrame(train)

In [25]:
# counting words in anchor column
wordcount_anchor = df['anchor'].str.split().str.len()
wordcount_anchor

0        1
1        1
2        1
3        1
4        1
        ..
36468    2
36469    2
36470    2
36471    2
36472    2
Name: anchor, Length: 36473, dtype: int64

In [26]:
wordcount_anchor.max()

5

In [27]:
# using the maximum number of words in 'anchor' plus one for the number of columns so each word in a row gets its own column
split_anchor = df['anchor'].str.rsplit(' ', 6, expand=True)
split_anchor

,0,1,2,3,4
0,abatement,None,None,None,None
1,abatement,None,None,None,None
2,abatement,None,None,None,None
3,abatement,None,None,None,None
4,abatement,None,None,None,None
...,...,...,...,...,...
36468,wood,article,None,None,None
36469,wood,article,None,None,None
36470,wood,article,None,None,None
36471,wood,article,None,None,None


In [28]:
# counting words in target column
wordcount_target = df['target'].str.split().str.len()
wordcount_target

0        3
1        3
2        2
3        2
4        2
        ..
36468    2
36469    2
36470    2
36471    2
36472    2
Name: target, Length: 36473, dtype: int64

In [29]:
wordcount_target.max()

15

In [30]:
# doing the same as we did for anchor
split_target = df['target'].str.rsplit(' ', 16, expand=True)
split_target

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,abatement,of,pollution,None,None,None,None,None,None,None,None,None,None,None,None
1,act,of,abating,None,None,None,None,None,None,None,None,None,None,None,None
2,active,catalyst,None,None,None,None,None,None,None,None,None,None,None,None,None
3,eliminating,process,None,None,None,None,None,None,None,None,None,None,None,None,None
4,forest,region,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,wooden,article,None,None,None,None,None,None,None,None,None,None,None,None,None
36469,wooden,box,None,None,None,None,None,None,None,None,None,None,None,None,None
36470,wooden,handle,None,None,None,None,None,None,None,None,None,None,None,None,None
36471,wooden,material,None,None,None,None,None,None,None,None,None,None,None,None,None
